# Case Study Model

This notebook shows the case study results supplement to the paper "*Risk- and Variance-Aware Electricity Pricing*" by Robert Mieth, Jip Kim and Yury Dvorkin.
Implementation and Models by Robert Mieth, building on AC-OPF and CC-ACOPF Code written by Miles Lubin and Yury Dvorkin available in https://github.com/dvorkinman/Code_for_CCACOPF

## General Remarks

- Framework
    - Requires Julia 1.1.
    - All necessary packages with their respective versions are indicated in the Julia project/manifest files 
    - The necessary packages are automatically installed *within the environment of this project*, i.e. executing this notebook will have no effect on globally installed packages and versions of your Julia distribution


In [24]:
## Instantiate Julia Environment
using Pkg
Pkg.activate(".")
# Pkg.instantiate()

"C:\\Users\\Robert\\Promotion\\Workshop\\02_cc_dlmp_varaware\\02_var_aware_accc_lmp\\_for_repo\\Project.toml"

In [25]:
## Load Packages
using Distributions, LinearAlgebra, SparseArrays, DataFrames, Dates # Basic Julia Utility
using CSV, JSON, JLD # For reading and writing data 
using JuMP # Modeling Framework
using Mosek, MosekTools # Solver and Solver Environment for conic programs
using Ipopt # Interior-point solver for non-convex ACOPF

ENV["Columns"] = 500 # Increase Default Column-Cutoff for better result display

In [26]:
## Load Functions and models
include("src/input.jl") # Type definitons and read-in functions
include("src/model_definitions.jl") # AC-OPF and CC AC-OPF Models
include("src/tools.jl") # Some additional functions
include("src/output.jl") # Postprocessing of solved model

save_results (generic function with 1 method)

In [27]:
## Load case data
casedat = load("casedata/118bus.jld")
# Note: Because MatpowerCases.jl has no Julia 1 support, 
# the data has been read with an older Julia version and pickled 
# in a .jld file to be read in Julia 1.

Dict{String,Any} with 4 entries:
  "generators"    => Generator[Generator(1, 1, 0.0, 0.0, 1.0, 0.0, 0.15, -0.05,…
  "buses"         => Bus[Bus(1, :PV, 0.561, 0.297, 0.0, 0.0, 0.0, 0.0, 1.0, 0.1…
  "lines"         => Line[Line(1, 2, 1, 0.0303, 0.0999, 2.7803, -9.16674, 0.0, …
  "generatorlist" => [1, 4, 6, 8, 10, 12, 15, 18, 19, 24  …  100, 103, 104, 105…

In [28]:
## Prepare Data
generators = casedat["generators"]
buses = casedat["buses"]
lines = casedat["lines"]
generatorlist = casedat["generatorlist"]

n_buses = length(buses)
n_lines = length(lines)
n_generators = length(generators)

refbus = 1
loadscale = 1.10
mvaBase = 100
thermalLimitscale = 0.6
theta_u = 15

line_limits= [175 175 500 175 175 175 500 500 500 175 175 175 175 175 175 175 175 175 175 175 500 175 175 175 175 175 175 175 175 175 500 500 500 175 175 500 175 500 175 175 140 175 175 175 175 175 175 175 175 500 500 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 500 175 175 500 500 500 500 500 500 500 175 175 500 175 500 175 175 500 500 175 175 175 175 175 175 175 500 175 175 175 175 175 175 500 500 175 500 500 200 200 175 175 175 500 500 175 175 500 500 500 175 500 500 175 175 175 175 175 175 175 175 175 175 200 175 175 175 175 175 175 175 175 175 500 175 175 175 175 175 175 175 175 175 175 175 175 175 175 175 500 175 175 175 500 175 175 175]

for i in 1:length(lines)
    lines[i].u = 0.99*thermalLimitscale * line_limits[i]/mvaBase
end

# Create Wind Farms
wp = 1.25 
factor_σ =  1.25*wp
voll = 10000

farms = Farm[]
push!(farms, Farm(70.0/100*wp,   factor_σ *7.0/100,  3))
push!(farms, Farm(147.0/100*wp,  factor_σ *14.7/100, 8))
push!(farms, Farm(102.0/100*wp,  factor_σ *10.2/100, 11))
push!(farms, Farm(105.0/100*wp,  factor_σ *10.5/100, 20))
push!(farms, Farm(113.0/100*wp,  factor_σ *11.3/100, 24))
push!(farms, Farm(84.0/100*wp,   factor_σ *8.4/100,  26))
push!(farms, Farm(59.0/100*wp,   factor_σ * 5.9/100, 31))
push!(farms, Farm(250.0/100*wp,  factor_σ *25.0/100, 38))
push!(farms, Farm(118.0/100*wp,  factor_σ *11.8/100, 43))
push!(farms, Farm(76.0/100*wp,   factor_σ *7.6/100,  49))
push!(farms, Farm(72.0/100*wp,   factor_σ *7.2/100,  53))
n_farms = length(farms)

for (i,f) in enumerate(farms)
    push!(buses[f.bus].farmids, i)
end

println("bus: ", n_buses)
println("line: ", n_lines)
println("gen: ", n_generators)
println("farm: ", n_farms)

PV = findall(b -> b.kind==:PV, buses)
PQ = findall(b -> b.kind==:PQ, buses)
REF = findall(b -> b.kind==:Ref, buses)
@assert length(REF) == 1

bus: 118
line: 186
gen: 54
farm: 11


In [29]:
## Run AC-OPF for linearization point
settings = Dict(
    "ϵ" => 0.01,
    "theta_u" => 180,
)
m_det = build_ac_opf(thermalLimitscale, generators, buses, lines, farms, settings);

solvetime = @elapsed optimize!(m_det)
objective_value(m_det)

>>> Total Deviaton = 1.4477201144585377
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     5156
Number of nonzeros in inequality constraint Jacobian.:     1358
Number of nonzeros in Lagrangian Hessian.............:     8252

Total number of variables............................:     1339
                     variables with only lower bounds:      183
                variables with lower and upper bounds:      186
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1167
Total number of inequality constraints...............:      825
        inequality constraints with only lower bounds:      227
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      598

iter    objective    inf_pr   inf_du lg(mu)  |

91082.92984624885

In [30]:
# Run Model with respective settings

# var_penalty = Dict("p_G" => 100, "q_G" => 100, "v" => 100, "f_p" => 100, "f_q" => 100)
var_penalty = Dict("p_G" => 0, "q_G" => 0, "v" => 0, "f_p" => 0, "f_q" => 0)

settings = Dict(
    # Set risk-parameter ϵ < 0.5
    "ϵ" => 0.01, 
    # Restrict voltage angle if numerical instability 
    # occurs (normally not necessary)
    "theta_u" => 180, 
    # "single": one alpha per generator (default); 
    #"split": one alpha per wind-farm and generator
    "alpha_mod" => "single", 
    # hand dict with variance penalty to model
    "var_penalty" => var_penalty, 
    # "det": no chance constraints; 
    # "gen_cc": chance constraints only on active power generation; 
    # "full_cc" all chance constraints
    "run_type" => "full_cc", 
)

m_cc, meta = build_ac_cc_opf(generators, buses, lines, farms, settings, m_det; print_output=false);
solvetime_cc = @elapsed optimize!(m_cc)
termination_status(m_cc)

MOSEK warning 705: #26 (nearly) zero elements are specified in sparse row ''(11767) of matrix 'A'.
MOSEK warning 705: #25 (nearly) zero elements are specified in sparse row ''(11768) of matrix 'A'.
MOSEK warning 705: #25 (nearly) zero elements are specified in sparse row ''(11769) of matrix 'A'.
MOSEK warning 705: #26 (nearly) zero elements are specified in sparse row ''(11770) of matrix 'A'.
MOSEK warning 705: #25 (nearly) zero elements are specified in sparse row ''(11771) of matrix 'A'.
MOSEK warning 705: #25 (nearly) zero elements are specified in sparse row ''(11772) of matrix 'A'.
MOSEK warning 705: #25 (nearly) zero elements are specified in sparse row ''(11773) of matrix 'A'.
MOSEK warning 705: #25 (nearly) zero elements are specified in sparse row ''(11774) of matrix 'A'.
MOSEK warning 705: #25 (nearly) zero elements are specified in sparse row ''(11775) of matrix 'A'.
MOSEK warning 705: #25 (nearly) zero elements are specified in sparse row ''(11776) of matrix 'A'.


OPTIMAL::TerminationStatusCode = 1

In [31]:
## Show alpha sum for sanity check
println(objective_value(m_cc))

if length(size((m_cc[:α]))) == 1
    sum(value.(m_cc[:α]))
else
    [sum(value.(m_cc[:α])[:,i]) for i in 1:size(m_cc[:α])[2]]
end

93139.61001278347


0.9999999910210383

In [34]:
## Process Results
res_dfs = jump_to_df(m_cc, settings, buses, generators, lines, farms; solvetime = solvetime_cc);

In [39]:
## Show Bus Results
display(res_dfs[1])

,bus,btype,haswind,gp,gq,v,alpha,std_v,std_q_G,std_p_G,lambda_p,lambda_q,gamma,delta_p_plus,delta_p_minus,delta_p_det_plus,delta_p_det_minus,mu_v_p,mu_v_m
,Int64,String,Bool,Real,Real,Float64,Float64,Float64,Float64,Real,Float64,Float64,Float64,Real,Real,Real,Real,Float64,Float64
1,1,PV,false,-1.37351e-7,0.05,1.01303,-3.0214e-9,0.0,0.0429859,-3.0214e-9,-2997.8,-16.9854,-26.327,-2.51184e-6,-247.894,-754.303,-2.47585e-6,-5.51683e-5,-3.44701e-5
2,2,PQ,false,0,0,1.00955,0.0,0.0123756,0.0,0,-3055.27,-11.4755,0.0,0,0,0,0,-0.000119467,-6.04806e-5
3,3,PQ,true,0,0,1.03128,0.0,0.00795715,0.0,0,-2934.05,-8.79722,0.0,0,0,0,0,-0.000254566,-3.4444e-5
4,4,PV,false,-1.38612e-7,2.91926,1.06,-3.36041e-9,0.0,0.0240501,-3.36041e-9,-2798.16,-9.58739e-5,0.0,-2.51185e-6,-256.816,-945.021,-2.47588e-6,-1567.52,-2.11484e-5
5,5,PQ,false,0,0,1.04349,0.0,0.00463671,0.0,0,-2776.09,-13.9151,0.0,0,0,0,0,-0.000447099,-2.73223e-5
6,6,PV,false,-1.37764e-7,0.497739,1.03228,-3.17813e-9,0.0,0.000971753,-3.17813e-9,-2942.87,-6.05959,0.0,-2.51184e-6,-251.926,-805.206,-2.47585e-6,-9.39483e-5,-2.73665e-5
7,7,PQ,false,0,0,1.02543,0.0,0.00923499,0.0,0,-2993.13,-4.96591,0.0,0,0,0,0,-0.000200187,-3.91866e-5
8,8,PV,true,-1.39667e-7,-2.79584,0.988486,-3.66626e-9,0.0,0.061243,-3.66626e-9,-2549.91,-4.01978e-5,0.0,-2.51184e-6,-294.668,-1155.43,-2.47586e-6,-3.56429e-5,-5.22259e-5
9,9,PQ,false,0,0,1.00481,0.0,0.0127838,0.0,0,-2529.43,-6.63343,0.0,0,0,0,0,-0.000101244,-7.0886e-5


In [40]:
## Show Line Results
display(res_dfs[2])

,f_p_ij,f_p_ji,f_q_ij,f_q_ji,std_pij,std_pji,std_qij,std_qji,aux_pij,aux_pji,aux_qij,aux_qji,eta_ij,eta_ji
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0247857,-0.02477,0.0154762,-0.04137,0.101326,0.101204,0.0988197,0.0986111,0.591416,0.59074,0.585873,0.587145,-2.79866e-5,-2.80311e-5
2,-0.585786,0.590696,-0.262476,0.267319,0.0544249,0.0537787,0.0701981,0.070073,0.776948,0.778723,0.52759,0.52891,-5.1249e-5,-5.226e-5
3,-0.744341,0.748815,2.36138,-2.34342,0.121881,0.123826,0.0572463,0.0597338,1.20154,1.21408,2.56235,2.55335,-3.86373e-5,-3.71404e-5
4,-0.240316,0.241172,-0.0796522,0.0529389,0.102531,0.103016,0.0917269,0.0923162,0.650612,0.65273,0.557147,0.553788,-3.09278e-5,-3.0847e-5
5,0.738634,-0.733072,0.0615122,-0.0516314,0.0415561,0.0421561,0.0622171,0.0631914,0.880154,0.877361,0.378911,0.38124,-6.63796e-5,-6.47394e-5
6,0.161072,-0.1609,0.307371,-0.312408,0.0898102,0.089586,0.0831834,0.0827706,0.573457,0.572147,0.647528,0.649776,-3.21176e-5,-3.2238e-5
7,-1.13639,1.13922,-0.978005,-0.138396,0.226373,0.24771,0.190948,0.244846,1.93985,2.05196,1.72675,1.45317,-1.43987e-5,-1.20439e-5
8,2.10275,-2.10275,-1.49326,1.6328,0.134935,0.129789,0.022657,0.00839417,2.43771,2.40469,1.58758,1.65233,-0.000107048,-23.4296
9,-1.13922,1.14316,0.138396,-1.30235,0.24771,0.207546,0.244846,0.155535,2.05196,1.86132,1.45317,1.88916,-1.20439e-5,-1.65647e-5


In [41]:
## Show Global Results
display(res_dfs[3])

,status,solvetime,objective,gen_cost,var_pen,sum_p_var,sum_q_var,sum_v_var,sum_fp_var,sum_fq_var
,MathOptI…,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,OPTIMAL,2.6149,93139.6,93139.6,0.0,0.0562679,9.29373,0.00646321,3.86991,3.26866


In [42]:
## Show Detailed Alhpa
display(res_dfs[4])

,gen,at_bus,alpha
,Int64,Int64,Float64
1,1,1,-3.0214e-9
2,2,4,-3.36041e-9
3,3,6,-3.17813e-9
4,4,8,-3.66626e-9
5,5,10,0.190868
6,6,12,5.88114e-7
7,7,15,2.28331e-8
8,8,18,1.88942e-8
9,9,19,-2.75385e-9


(c) 2019, Robert Mieth, robert.mieth@ieee.org